# Clifford+RZ and Hardware Compilation

You need the `pyqrack` package to run this notebook. [`vm6502q/pyqrack`](https://github.com/vm6502q/pyqrack) is a pure Python wrapper on the [`vm6502q/qrack`](https://github.com/vm6502q/qrack) quantum computer simulation framework core library. The preferred method of installation is from source code, at those GitHub repositories, but a package with default build precompiled binaries is available on [pypi](https://pypi.org/project/pyqrack/0.2.0/).

In [1]:
# For example, if your Jupyter installation uses pip:
# import sys
# !{sys.executable} -m pip install --platform=manylinux2014_x86_64 --only-binary=:all: pyqrack

[`QrackSimulator`](https://github.com/vm6502q/pyqrack/blob/main/pyqrack/qrack_simulator.py) is the "workhorse" of the `pyqrack` package. It instantiates simulated "registers" of qubits that we can act basic quantum gates between, to form arbitrary universal quantum circuits.

[`QrackCircuit`](https://github.com/vm6502q/pyqrack/blob/main/pyqrack/qrack_circuit.py) is an optional class for optimizing compilation. With it, one can define a circuit in advance, which is optimized upon definition. Then, one can save the optimized result to a file and later load it into a new or existing `QrackCircuit` instance. Ultimately, the circuit is executed by calling `run()` on a `QrackCircuit`, with a parameter of `QrackSimulator` of appropriate size. (The necessary width of the `QrackSimulator` can be determined with `QrackCircuit.get_qubit_count()`.)

In [2]:
import collections
import copy
import math
import os
import random
# import warnings

import numpy as np

import tensorcircuit as tc
import tensorcircuit.compiler.simple_compiler as tcsc

from qiskit import QuantumCircuit
from qiskit.compiler import transpile

from pyqrack import QrackSimulator, QrackCircuit

# If we disable OpenCL and set the max CPU qubits to max integer, we bypass qubit widths limits.
os.environ["QRACK_MAX_CPU_QB"]="-1"

In [3]:
width = 6
max_magic = 6

We're going to try to optimize the compilation of a quantum volume circuit for minimum gate count and depth of critical path.

In [4]:
def rand_1qb(circ, q):
    th = random.uniform(0, 4 * math.pi)
    ph = random.uniform(0, 4 * math.pi)
    lm = random.uniform(0, 4 * math.pi)
    circ.u(th, ph, lm, q)


def random_circuit(width, circ):
    for n in range(width):
        # Single bit gates
        for j in range(width):
            rand_1qb(circ, j)

        # Multi bit gates
        bit_set = [i for i in range(width)]
        while len(bit_set) > 1:
            b1 = random.choice(bit_set)
            bit_set.remove(b1)
            b2 = random.choice(bit_set)
            bit_set.remove(b2)
            circ.cx(b1, b2)

    return circ

In [5]:
orig_circ = random_circuit(width, QuantumCircuit(width))

print("Raw gate count: ", sum(dict(orig_circ.count_ops()).values()))
print("Raw depth of critical path: ", orig_circ.depth())
print("Raw qubit width: ", orig_circ.width())

Raw gate count:  54
Raw depth of critical path:  12
Raw qubit width:  6


If we have Qiskit and numpy installed, we can transpile to a near-Clifford Qiskit circuit.

In [6]:
basis_gates=["rz", "x", "y", "z", "sx", "sy", "s", "sdg", "cx", "cy", "cz", "swap", "iswap", "iswap_dg"]
circ = transpile(orig_circ, basis_gates=basis_gates, optimization_level=3)

print("Raw (near-Clifford) gate count: ", sum(dict(circ.count_ops()).values()))
print("Raw (near-Clifford) depth of critical path: ", circ.depth())
print("Raw (near-Clifford) qubit width: ", circ.width())

Raw (near-Clifford) gate count:  164
Raw (near-Clifford) depth of critical path:  34
Raw (near-Clifford) qubit width:  6


From there, we can simulate with Qrack's near-Clifford-optimized simulation layer, everything except for measurement sampling. We write the resulting "intermediate representation" into a file on disk.

In [7]:
qsim = QrackSimulator(width, isSchmidtDecomposeMulti=False, isSchmidtDecompose=False, isOpenCL=False)
qsim.run_qiskit_circuit(circ, 0)
qsim.out_to_file("qrack_circuit.chp")
qsim.reset_all()
qsim = None

Then, we read this into a new Qiskit circuit, which is still near-Clifford. The gate count and qubit width might be higher, but the circuit is entirely Clifford group, except for a single terminal layer of non-Clifford gates, (followed by either post selection or a rudimentary error-correction routine).

In [8]:
circ = QrackSimulator.file_to_optimized_qiskit_circuit("qrack_circuit.chp")

if circ.width() <= 16:
    print(circ)
print("(Near-Clifford) gate count: ", sum(dict(circ.count_ops()).values()))
print("(Near-Clifford) depth of critical path: ", circ.depth())
print("(Near-Clifford) qubit width: ", circ.width())

      ┌─────────────────────────┐                                    »
q_0: ─┤ U(2.0041,1.3337,-2.093) ├────────────────────────────────────»
      ├─────────────────────────┴┐      ┌──────────────────────┐     »
q_1: ─┤ U(1.3834,1.7056,-2.2001) ├───■──┤ U(π/2,2.0049,2.9629) ├──■──»
     ┌┴──────────────────────────┴┐  │  └──────────────────────┘┌─┴─┐»
q_2: ┤ U(0.55354,-1.3865,0.93538) ├──┼──────────────────────────┤ X ├»
     └┬──────────────────────────┬┘  │                          └───┘»
q_3: ─┤ U(2.3062,2.3912,-2.3543) ├───┼───────────────────────────────»
     ┌┴──────────────────────────┤   │                               »
q_4: ┤ U(0.82988,-1.6696,2.0601) ├───┼───────────────────────────────»
     ├───────────────────────────┴┐┌─┴─┐┌──────────────────────┐     »
q_5: ┤ U(2.5014,0.61276,-0.24735) ├┤ X ├┤ U(0.10503,-π/2,-π/2) ├─────»
     └────────────────────────────┘└───┘└──────────────────────┘     »
«                                            ┌──────────────────┐         »
«

`QrackCircuit` might reduce the gate count and depth further, but it's probably an unrealistic gate set for most gate-based quantum hardware. However, if we're using conventional Qrack simulation, rather than near-Clifford, this format is advantageous for native Qrack.

In [9]:
qcircuit = QrackCircuit.in_from_qiskit_circuit(circ)
qcircuit.out_to_file("qrack_circuit.qc")
qrack_circ = QrackCircuit.file_to_qiskit_circuit("qrack_circuit.qc")

if qrack_circ.width() <= 16:
    print(qrack_circ)
print("QrackCircuit gate count: ", sum(dict(qrack_circ.count_ops()).values()))
print("QrackCircuit depth of critical path: ", qrack_circ.depth())
print("QrackCircuit qubit width: ", qrack_circ.width())

     ┌─────────────┐                                               »
q_0: ┤ Multiplexer ├───────────────────────────────────────────────»
     ├─────────────┤┌──────────────┐┌─────────────┐┌──────────────┐»
q_1: ┤ Multiplexer ├┤1             ├┤ Multiplexer ├┤1             ├»
     ├─────────────┤│              │└─────────────┘│  Multiplexer │»
q_2: ┤ Multiplexer ├┤              ├───────────────┤0             ├»
     ├─────────────┤│              │               └──────────────┘»
q_3: ┤ Multiplexer ├┤  Multiplexer ├───────────────────────────────»
     ├─────────────┤│              │                               »
q_4: ┤ Multiplexer ├┤              ├───────────────────────────────»
     ├─────────────┤│              │                               »
q_5: ┤ Multiplexer ├┤0             ├───────────────────────────────»
     └─────────────┘└──────────────┘                               »
«                    ┌──────────────┐┌─────────────┐┌──────────────┐»
«q_0: ───────────────┤1          

We might further optimize with tensor network software.

In [10]:
tc.set_backend("tensorflow")
tc.set_contractor("auto")
tc.set_dtype("complex64")

net = tc.Circuit.from_qiskit(circ)
net = tcsc.simple_compile(net)[0]
net.draw()

┌─────────────┐ ┌───┐ ┌────────────┐┌───┐  ┌────────────┐      »
q_0: ┤ Rz(-3.6638) ├─┤ H ├─┤ Rz(2.0041) ├┤ H ├──┤ Rz(2.9045) ├──────»
     ├─────────────┤ ├───┤ ├────────────┤├───┤  ├────────────┤      »
q_1: ┤ Rz(-3.7709) ├─┤ H ├─┤ Rz(1.3834) ├┤ H ├──┤ Rz(3.2764) ├───■──»
     ├─────────────┴┐├───┤┌┴────────────┤├───┤ ┌┴────────────┤   │  »
q_2: ┤ Rz(-0.63541) ├┤ H ├┤ Rz(0.55354) ├┤ H ├─┤ Rz(0.18429) ├───┼──»
     ├─────────────┬┘├───┤└┬────────────┤├───┤ └┬───────────┬┘   │  »
q_3: ┤ Rz(-3.9251) ├─┤ H ├─┤ Rz(2.3062) ├┤ H ├──┤ Rz(3.962) ├────┼──»
     ├─────────────┤ ├───┤┌┴────────────┤├───┤┌─┴───────────┴─┐  │  »
q_4: ┤ Rz(0.48927) ├─┤ H ├┤ Rz(0.82988) ├┤ H ├┤ Rz(-0.098851) ├──┼──»
     ├─────────────┤ ├───┤└┬────────────┤├───┤└─┬────────────┬┘┌─┴─┐»
q_5: ┤ Rz(-1.8181) ├─┤ H ├─┤ Rz(2.5014) ├┤ H ├──┤ Rz(2.1836) ├─┤ X ├»
     └─────────────┘ └───┘ └────────────┘└───┘  └────────────┘ └───┘»
«                                                                »
«q_0: ───────────────────────────────────────────────────────────»
«      ┌────────────┐┌───┐ ┌────────────┐┌───┐┌────────────┐     »
«q_1: ─┤ Rz(1.3921) ├┤ H ├─┤ Rz(1.5708) ├┤ H ├┤ Rz(3.5757) ├──■──»
«      └────────────┘└───┘ └────────────┘└───┘└────────────┘┌─┴─┐»
«q_2: ──────────────────────────────────────────────────────┤ X ├»
«                                                           └───┘»
«q_3: ───────────────────────────────────────────────────────────»
«                                                                »
«q_4: ───────────────────────────────────────────────────────────»
«     ┌─────────────┐┌───┐┌─────────────┐┌───┐                   »
«q_5: ┤ Rz(-3.1416) ├┤ H ├┤ Rz(0.10503) ├┤ H ├───────────────────»
«     └─────────────┘└───┘└─────────────┘└───┘                   »
«                                                                 »
«q_0: ─────────────────────────────────────────────────────────■──»
«     ┌──────────────┐┌───┐┌────────────┐┌───┐ ┌────────────┐  │  »
«q_1: ┤ Rz(-0.43411) ├┤ H ├┤ Rz(1.5708) ├┤ H ├─┤ Rz(3.8989) ├──┼──»
«     ├─────────────┬┘├───┤├────────────┤├───┤┌┴────────────┤┌─┴─┐»
«q_2: ┤ Rz(-4.4059) ├─┤ H ├┤ Rz(2.2448) ├┤ H ├┤ Rz(0.46927) ├┤ X ├»
«     └─────────────┘ └───┘└────────────┘└───┘└─────────────┘└───┘»
«q_3: ────────────────────────────────────────────────────────────»
«                                                                 »
«q_4: ────────────────────────────────────────────────────────────»
«                                                                 »
«q_5: ────────────────────────────────────────────────────────────»
«                                                                 »
«     ┌──────────────┐┌───┐┌────────────┐┌───┐┌────────────┐     »
«q_0: ┤ Rz(-0.67428) ├┤ H ├┤ Rz(1.5708) ├┤ H ├┤ Rz(1.5708) ├──■──»
«     └──────────────┘└───┘└────────────┘└───┘└────────────┘  │  »
«q_1: ────────────────────────────────────────────────────────┼──»
«     ┌─────────────┐ ┌───┐┌────────────┐┌───┐┌───────────┐   │  »
«q_2: ┤ Rz(-1.5384) ├─┤ H ├┤ Rz(1.4019) ├┤ H ├┤ Rz(4.522) ├───┼──»
«     └─────────────┘ └───┘└────────────┘└───┘└───────────┘   │  »
«q_3: ────────────────────────────────────────────────────────┼──»
«                                                             │  »
«q_4: ────────────────────────────────────────────────────────┼──»
«                                                           ┌─┴─┐»
«q_5: ──────────────────────────────────────────────────────┤ X ├»
«                                                           └───┘»
«          ┌───┐     ┌────────────┐    ┌───┐     ┌────────────┐     »
«q_0: ─────┤ H ├─────┤ Rz(1.5708) ├────┤ H ├─────┤ Rz(3.4147) ├─────»
«          └───┘     └────────────┘    └───┘     └────────────┘     »
«q_1: ──────────────────────────────────────────────────────────────»
«                                                                   »
«q_2: ───────────────────────────────────────────────────────────■──»
«                                                

We can sample with tensorcircuit at this point, or we can try another round of optimization.

In [11]:
%%time
print(net.sample(allow_state=True, batch=256, format="count_dict_bin"))

{'000000': 1, '000001': 2, '000100': 7, '000101': 10, '000111': 2, '001100': 5, '001110': 1, '010000': 1, '010001': 2, '010100': 8, '010101': 22, '010110': 3, '010111': 2, '011000': 6, '011001': 3, '011010': 2, '011100': 19, '011101': 26, '011110': 2, '011111': 2, '100001': 2, '100100': 16, '100101': 25, '100110': 1, '100111': 3, '101001': 7, '101100': 22, '101101': 15, '101110': 1, '101111': 5, '110000': 1, '110001': 1, '110101': 17, '110111': 1, '111001': 3, '111010': 1, '111100': 3, '111101': 6}
CPU times: user 100 ms, sys: 4.02 ms, total: 104 ms
Wall time: 102 ms


In [12]:
circ = net.to_qiskit()
basis_gates=["rz", "x", "y", "z", "sx", "sy", "s", "sdg", "cx", "cy", "cz", "swap", "iswap", "iswap_dg"]
circ = transpile(circ, basis_gates=basis_gates, optimization_level=3)

In [13]:
def cross_entropy(measurement_list, qiskit_circuit):
    perm_count = 1 << qiskit_circuit.width()
    ideal_shots = perm_count << 12

    sim = QrackSimulator(qubitCount = qiskit_circuit.width(), isStabilizerHybrid=False)
    sim.run_qiskit_circuit(qiskit_circuit, 0)
    ideal_result = sim.measure_shots(list(range(qiskit_circuit.width())), ideal_shots)
    ideal_result = dict(collections.Counter(ideal_result))
    for key, value in ideal_result.items():
        ideal_result[key] = value / ideal_shots
        
    ideal_probs_sv = []
    for i in range(perm_count):
        ideal_probs_sv.append(ideal_result[i] if i in ideal_result else 0)
    
    ideal_probs = []
    for bit_string in measurement_list:
        ideal_probs.append(ideal_probs_sv[bit_string])
                               
    return (perm_count * np.mean(ideal_probs)) - 1


def hellinger_fidelity(measurement_list, qiskit_circuit):
    perm_count = 1 << qiskit_circuit.width()
    ideal_shots = perm_count << 12

    t_histogram = collections.Counter(measurement_list)
    shot_count = sum(t_histogram.values())
    t_histogram = dict(t_histogram)
        
    histogram = {}
    for key, value in t_histogram.items():
        histogram[key] = value / shot_count

    sim = QrackSimulator(qubitCount = qiskit_circuit.width(), isStabilizerHybrid=False)
    sim.run_qiskit_circuit(qiskit_circuit, 0)
    ideal_result = sim.measure_shots(list(range(qiskit_circuit.width())), ideal_shots)
    ideal_result = dict(collections.Counter(ideal_result))
    for key, value in ideal_result.items():
        ideal_result[key] = value / ideal_shots
        
    ideal_probs_sv = []
    for i in range(perm_count):
        ideal_probs_sv.append(ideal_result[i] if i in ideal_result else 0)

    fidelity = 0
    for qubit_permutation in histogram.keys():
        ideal_prob = ideal_result[qubit_permutation] if qubit_permutation in ideal_result else 0
        normalized_frequency = histogram[qubit_permutation]
        fidelity += math.sqrt(ideal_prob * ideal_prob)
    fidelity *= fidelity
                               
    return fidelity


if width <= 16:
    # The circuit has 0 ancillae, so we can generate samples with conventional simulation.
    qsim = QrackSimulator()
    qcircuit.run(qsim)
    for i in range(width, qcircuit.get_qubit_count()):
        qsim.force_m(i, False)
    shots = 1 << (width + 12)
    samples = qsim.measure_shots(list(range(width)), shots)
    # print("XEB fidelity: ", cross_entropy(samples, orig_circ))
    print("Hellinger fidelity: ", hellinger_fidelity(samples, orig_circ))

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_UHD_Graphics_[0x9bc4].ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
Hellinger fidelity:  1.0
